Connected to Python 3.9.10

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import pickle
from openpyxl import Workbook  
import datetime
import importlib as imp
import scipy.optimize as so
import scipy.sparse as sp
from pylatex import Document, Section, Subsection, Command, Package
from tabulate import tabulate
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import time as time
# import cProfile

import time
# For "converter" to convert strings stored with .csv to numpy arrays
# https://stackoverflow.com/questions/42755214/how-to-keep-numpy-array-when-saving-pandas-dataframe-to-csv

import ast   # Abstract Syntax Trees - used in the "converter" for reading .csv
#%% Import py files
## Fixing Paths

sys.path.append('../../src/package')
sys.path.append('../../../BondsTable')
sys.path.append('../../tests')
sys.path.append('../../data')
sys.path.append('../../../FORTRAN2024/code')
OUTPUT_DIR = '../../output'

In [2]:
import DateFunctions_1 as dates
import pvfn as pv
import pvcover as pvc
import discfact as discfact
import calculate_ratesprices as outputs
#import CRSPBondsAnalysis as analysis

import parzeroBond as pzb
import crsp_data_processing as data_processing
import produce_inputs as inputs
import util_fn as util
import output_to_latexpdf as output_to
imp.reload(dates)
imp.reload(pv)
imp.reload(pvc)
imp.reload(inputs)
imp.reload(outputs)
imp.reload(pzb)
imp.reload(util)
imp.reload(output_to)
#%% Wrapper Function

def produce_curve_wrapper(bonddata, curvetypes, start_date, end_date, breaks, filename, calltype=0,
                          wgttype=1, lam1=1, lam2=2, padj=False, padjparm=0, yield_to_worst=False, tax=False,
                          yvolsflg=False, yvols=0.1):
    """Loop through months from a given start and end date for cleaned crsp UST bond data to produce a df of a series of estimate parms 
    --forward rates, yvols, and a df of predicted versus actual price and yield. 
    Args:
        bonddata (pd.DataFrame): Cleaned CRSP bond dataset, after applying data_processing.clean_crsp and
                                 data_processing.create_weight to the WRDS CRSP monthly UST data.
        curvetypes (list): A list of curve type strings.
        start_date (int): Start date for the end result, in the format YYYYMMDD.
        end_date (int): End date for the end result, in the format YYYYMMDD.
        breaks (np.array): An array of date breaks, in years, e.g., np.array([0.0833, 0.5, 1., 2., 5., 10., 20., 30.])
        filename (str): The base filename for the output files.
        calltype (int, optional): Whether to filter callable bonds. 0=all, 1=callable only, 2=non-callable only.
                                  Defaults to 0.
        wgttype (int, optional): The weight type for SSQ. Defaults to 1.
        lam1 (float, optional): Lambda parameter 1 for weighting. Defaults to 1.
        lam2 (float, optional): Lambda parameter 2 for weighting. Defaults to 2.
        padj (bool, optional): Whether to apply price adjustment. Defaults to False.
        padjparm (float, optional): Parameter for price adjustment. Defaults to 0.
        yield_to_worst (bool, optional): Whether to use yield to worst for filtering. Defaults to False.
        tax (bool, optional): Whether to calculate and include tax-adjusted curves. Defaults to False.
        yvolsflg (bool, optional): Whether to include yield volatilities. Defaults to False.
        yvols (float, optional): Yield volatilities parameter. Defaults to 0.1.
    Returns:
        - final_curve_df: DataFrame with final curve/estimated data.
        - final_price_yield_df: DataFrame with final price and yield data.
    """
    # Convert start_date and end_date to datetime format if they are not already
    if not isinstance(start_date, datetime.datetime):
        start_date = pd.to_datetime(str(start_date), format='%Y%m%d')
    if not isinstance(end_date, datetime.datetime):
        end_date = pd.to_datetime(str(end_date), format='%Y%m%d')
    # Dictionary to store final DataFrames for each curve type
    #all_dfs = {}
    curve_data_list = []
    price_yield_data_list = []
    wb = Workbook()
    for curvetype in curvetypes:
        # curvetype = 'pwtf'
        #curve_data_list = []
        curve_tax_data_list = []
        #price_yield_data_list = []
        filtered_data = bonddata[(bonddata['quote_date'] > start_date) & (bonddata['quote_date'] < end_date)]
        quotedates = list(set(filtered_data['MCALDT']))
        for quotedate in quotedates:
            # quotedate=20000929
            parms = inputs.read_and_process_csvdata(filtered_data, quotedate, calltype)
            quotedate = int(quotedate)
            quotedate_Julian = dates.YMDtoJulian(quotedate)[0]
            parms = inputs.filter_yield_to_worst_parms(quotedate, parms, yield_to_worst)
            parms = inputs.create_weight(parms, wgttype, lam1=lam1, lam2=lam2)
            len_parms = len(parms)
            curve, prices, bondpv, stderr, yvol, mesg, ier = outputs.calc_rate_notax(parms, quotedate, breaks, curvetype, wgttype, lam1, lam2,
                                                                   padj, padjparm, yvolsflg=yvolsflg, yvols=yvols)
            curve.append(stderr)  # add std errors
            curve.append(yvol)
            curve.append(mesg)
            curve.append(ier)
            parms['YTM'] = (parms['Maturity Date at time of Issue'] - quotedate_Julian)/365.25
            # Count the number of bonds in each forward period
            def classify_ytm(ytm, breaks):
                for i in range(len(breaks) - 1):
                    if breaks[i] < ytm <= breaks[i + 1]:
                        return breaks[i]
                if ytm <= breaks[0]:
                    return 0
                if ytm > breaks[-1]:
                    return breaks[-1]
                return None
            parms['YTM_bucket'] = parms['YTM'].apply(lambda x: classify_ytm(x, breaks))
            bucket_counts = parms['YTM_bucket'].value_counts().reindex(breaks, fill_value=0).sort_index()
            num_bonds = bucket_counts.tolist()
            curve.append(np.array(num_bonds))  # add number of bonds in each forward period
            price_yield_df = outputs.get_predicted_actual_yieldprice_notax(parms, bondpv, prices, quotedate,
                curvetype, padj, yvolsflg=yvolsflg, yvols=yvol)
            price_yield_df.insert(0, 'QuoteDate', quotedate)
            price_yield_df.insert(0, 'type', curvetype)
            # Make multi-index
            price_yield_df.set_index(['type','QuoteDate'],inplace=True, drop=False)
            # Aggregate curve and price_yield_df data
            #curve_df = pd.DataFrame(curve[3].reshape(1, -1), index=[quotedate], columns=breaks)
            #curve_data_list.append(curve_df)
            # Change from Kathy's - build up list of full curves (not just rates)
            curve_data_list.append(curve)
            price_yield_data_list.append(price_yield_df)
            print(quotedate)
        #final_curve_df = pd.concat(curve_data_list)
        # Change from Kathy's - make list into dataframe, then define columns, sort
        #final_curve_df = final_curve_df.sort_index()
        #final_price_yield_df = pd.concat(price_yield_data_list)
        #final_price_yield_df = final_price_yield_df.sort_values(by=['QuoteDate', 'MatYr', 'MatMth', 'MatDay'])
        #all_dfs[curvetype] = {'curve_df': final_curve_df,
#                              'price_yield_df': final_price_yield_df}
# NB - need to fix up tax
        if tax:
            tax_spd = np.array(["tax2_spd", "tax3_spd"])
            curve_tax_df = pd.DataFrame(curve_tax[3].reshape(1, -1), index=[quotedate], columns=np.append(breaks, tax_spd))
            curve_tax_data_list.append(curve_tax_df)
            curve_tax = outputs.cal_rate_with_tax(parms, quotedate, breaks, curvetype, wgttype, lam1, lam2, padj, padjparm)
            final_curve_tax_df = pd.concat(curve_tax_data_list)
            final_curve_tax_df = final_curve_tax_df.sort_index()
            all_dfs[curvetype]['curve_tax_df'] = final_curve_tax_df
    # Convert the accumulated list of curves into a df and sort by index 
    # The following is necessary because we want to have a multi-index (on 
    # quotedate and curve type) but the quote date is a np.ndarray and 
    # multi-index on that does not work. So:
        # Create a new column and name it 'quotedate_ind'
        # Convert to integer and then create index
        # Also create type_ind, so that we retain the original type and quotedate in the df
    final_curve_df = pd.DataFrame(curve_data_list)
    final_curve_df.columns = ['type','quotedate_jul','breaks','rates', 'stderr', 'yvols', 'mesg', 'ier', 'num_bonds']
    final_curve_df['ytw_flag'] = yield_to_worst
    final_curve_df['padj'] = padj
    final_curve_df['padjparm'] = padjparm
    if tax == False:  # ?? tax status
        final_curve_df['taxability'] = 0
    final_curve_df['quotedate_ymd'] = final_curve_df['quotedate_jul']
    final_curve_df['quotedate_ymd'] = final_curve_df['quotedate_ymd'].map(dates.JuliantoYMDint)
    final_curve_df['quotedate_ymd'] = final_curve_df['quotedate_ymd'].map(int)
    final_curve_df['type_ind'] = final_curve_df['type']
    final_curve_df['quotedate_ind'] = final_curve_df['quotedate_ymd']
    final_curve_df.set_index(['type_ind','quotedate_ind'],inplace=True,drop=True)
    final_curve_df = final_curve_df.sort_index()
    # Converted accumulate list of actual vs predicted into df
    final_price_yield_df = pd.concat(price_yield_data_list)
    final_price_yield_df = final_price_yield_df.sort_values(by=[ 'MatYr', 'MatMth', 'MatDay'])
    final_price_yield_df = final_price_yield_df.sort_index()
    # Export to CSV
    # final_curve_df.to_csv(os.path.join(OUTPUT_DIR, f'curve_{filename}.csv'))
    if tax:
        final_curve_tax_df.to_csv(os.path.join(OUTPUT_DIR, f'curve_tax_{filename}.csv'))
    final_price_yield_df.to_csv(os.path.join(OUTPUT_DIR, f'price_yield_{filename}.csv'))
    # Export to Excel on separate sheets
    excel_file_path = os.path.join(OUTPUT_DIR, f'{filename}_data.xlsx')
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        final_curve_df.to_excel(writer, sheet_name='Curve Data')
        if tax:
            final_curve_tax_df.to_excel(writer, sheet_name='Curve Tax Data')
        final_price_yield_df.to_excel(writer, sheet_name='Price Yield Data')
#    return all_dfs
    return final_curve_df, final_price_yield_df
#%% Produce and output par bond, zero bond, and annuity rates and price tables

def pb_zb_anty_wrapper(curve_df, table_breaks_yr, estfile, twostep=True, parmflag=True, padj=False):
    """Produce par bond, zero bond, annuity rates and prices tables. """
# Create df of bond parameters
    parms_df = pzb.create_parbnd_df(curve_df, table_breaks_yr, addmths=False)
    # Produce pb, zb, annuity prices and rates tables
    # For the par bonds take back the updated parms_df which has the par bond coupons inserted
    # For zero and annuity just throw away the returned parms_df because it is unchanged
    parbd_rateprice_df, pbparms_df = pzb.produce_pb_zb_anty_dfs(curve_df, parms_df,
     'parbond', twostep, parmflag, padj)
    zerobd_rateprice_df, x1 = pzb.produce_pb_zb_anty_dfs(curve_df, parms_df,
     'zerobond', parmflag, padj)
    annuity_rateprice_df, x1 = pzb.produce_pb_zb_anty_dfs(curve_df, parms_df,
     'annuity', parmflag, padj)
# Put par bond coupons into the parms_df
     # Export to CSV
    dataframes = [parbd_rateprice_df, zerobd_rateprice_df, annuity_rateprice_df]
    names = ['parbd_rates_prices', 'zerobd_rates_prices', 'annuity_rates_prices']
    util.export_to_csv(dataframes, names, OUTPUT_DIR, estfile)
    return parbd_rateprice_df, zerobd_rateprice_df, annuity_rateprice_df, pbparms_df

def seperate_pb_zb_anty_wrapper(parbd_rateprice_df, zerobd_rateprice_df, annuity_rateprice_df):
    parbd_rate = parbd_rateprice_df.xs('rate', level='rtype')
    parbd_cprice = parbd_rateprice_df.xs('cprice', level='rtype')
    zerobd_rate = zerobd_rateprice_df.xs('rate', level='rtype')
    zerobd_cprice = zerobd_rateprice_df.xs('cprice', level='rtype')
    annuity_rate = annuity_rateprice_df.xs('rate', level='rtype')
    annuity_cprice = annuity_rateprice_df.xs('cprice', level='rtype')
    return parbd_rate, parbd_cprice, zerobd_rate, zerobd_cprice, annuity_rate, annuity_cprice

def seperate_returns_wrapper(ret_df):
    total_ret = ret_df.xs('total_return', level='return_type')
    income_ret = ret_df.xs('income_return', level='return_type')
    capgain_ret = ret_df.xs('capgain_return', level='return_type')
    return total_ret, income_ret, capgain_ret

In [3]:
filepath = '../../data/USTMonthly.csv'
#filepath = 'C:/Users/zhang/OneDrive/Documents/GitHub/UST-yieldcurves_2024/curve_utils/data/USTMonthly.csv'
## Define user inputs

estfile = '1986to2000'
estfile = 'testshort'
estfile = 'test2000opt_030'
estfile = 'test2000opt_015'
estfile = 'test1990opt_vol_pwtf'
estfile = 'test2000opt_vol'
yvolsflg = True  # to estimate
yvols = 0.2
yield_to_worst = False # False - w/opt
tax = False
calltype = 0  # 0 to keep all bonds, 1 for callable bonds, 2 for non-callable bonds
curvetypes =  ['pwcf', 'pwlz', 'pwtf'] # ['pwtf']  #
start_date = 19860701
start_date = 19800101
end_date = 19830101
start_date = 19900101
end_date = 19950101
start_date = 20000101
end_date = 20020101

In [4]:
plot_points_yr = np.arange(0,32,.01)  # np.arange(.01,4,.01), np.arange(4.01,32,.01)
table_breaks_yr = np.array([0.0833, 0.5, 1, 1.5, 2, 3, 4, 5, 7, 10, 15, 20, 30])
wgttype=1
lam1=1
lam2=2
sqrtscale=True
twostep = True
parmflag = True
padj = False
padjparm = 0
# TSC 27-apr-2024 Trying various sets of breaks - problems with too fine at short end

breaks = np.array([7/365.25, 14/365.25, 21/365.25, 28/365.25, 35/365.25, 52/365.25, 92/365.25, 
                   184/365.25, 1, 2, 4, 8, 16, 24, 32])  # np.array([0.0833, 0.5, 1.,2.,5.,10.,20.,30.])

breaks = np.array([14/365.25, 28/365.25, 52/365.25, 92/365.25, 
                   184/365.25, 1, 2, 4, 8, 16, 24, 32])  # np.array([0.0833, 0.5, 1.,2.,5.,10.,20.,30.])

breaks = np.array([28/365.25, 92/365.25, 
                   184/365.25, 1, 2, 4, 8, 16, 24, 32])  # np.array([0.0833, 0.5, 1.,2.,5.,10.,20.,30.])

curve_points_yr1 = np.arange(.01,4,.01)
curve_points_yr2 = np.arange(4.01,32,.01)
curve_points_yr3 = np.arange(0,32,.01)

In [5]:
df_curve = pd.read_csv(OUTPUT_DIR+'/'+estfile+'_curve.csv',index_col=[0,1],
                   converters={'quotedate':util.from_np_array,'breaks': util.from_np_array,'rates': util.from_np_array})
# May be just easier to write & read pickle, but .csv is humanly readable and transferrable

df_curve = pd.read_pickle(OUTPUT_DIR+'/'+estfile+'_curve.pkl')
df_price_yield = pd.read_pickle(OUTPUT_DIR+'/'+estfile+'_predyld.pkl')

In [6]:
parbd_rateprice_df, zerobd_rateprice_df, annuity_rateprice_df, pbparms_df = pb_zb_anty_wrapper(
    df_curve, table_breaks_yr, estfile, twostep=twostep, parmflag=parmflag, padj=padj)

parbd_rate, parbd_cprice, zerobd_rate, zerobd_cprice, annuity_rate, annuity_cprice = seperate_pb_zb_anty_wrapper(
    parbd_rateprice_df, zerobd_rateprice_df, annuity_rateprice_df)

c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,4] = "A/A"
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'eomyes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,5] = 'eomyes'
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly c

In [7]:
ret_df = pzb.return_wrapper(df_curve, table_breaks_yr, twostep=False, parmflag=True, padj=False)

c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,4] = "A/A"
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'eomyes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,5] = 'eomyes'
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly c

ValueError: Index mismatch between total return and income return dataframes.

In [8]:
total_ret = pzb.calc_par_total_ret(curve_df, table_breaks_yr, twostep, parmflag, padj)
income_ret_df = pzb.calc_par_income_ret(curve_df, table_breaks_yr, twostep, parmflag, padj)

NameError: name 'curve_df' is not defined

In [9]:
curve_df = df_curve.copy()

In [10]:
total_ret = pzb.calc_par_total_ret(curve_df, table_breaks_yr, twostep, parmflag, padj)
income_ret_df = pzb.calc_par_income_ret(curve_df, table_breaks_yr, twostep, parmflag, padj)

c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,4] = "A/A"
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'eomyes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,5] = 'eomyes'
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly c

In [11]:
income_ret_df

0.5YR     1.0YR     1.5YR     2.0YR  \
ctype quotedate  return_type                                             
pwcf  20000131.0 income_return  0.004947  0.005361  0.005526  0.005609   
      20000229.0 income_return  0.004705  0.004937  0.005092  0.005169   
      20000331.0 income_return  0.005196  0.005361  0.005444  0.005444   
      20000428.0 income_return  0.005108  0.005348  0.005428  0.005428   
      20000531.0 income_return  0.005361  0.005692  0.005609  0.005609   
...                                  ...       ...       ...       ...   
pwtf  20010831.0 income_return  0.002866  0.002866  0.002949  0.003116   
      20010928.0 income_return  0.002043  0.002124  0.002205  0.002368   
      20011031.0 income_return  0.001606  0.001690  0.001859  0.002027   
      20011130.0 income_return  0.001473  0.001717  0.002043  0.002368   
      20011231.0 income_return  0.001438  0.001690  0.002195  0.002614   

                                   3.0YR     4.0YR     5.0YR     7.0YR  \
ctype quotedate  return_type                                             
pwcf  20000131.0 income_return  0.005609  0.005692  0.005692  0.005692   
      20000229.0 income_return  0.005246  0.005246  0.005246  0.005246   
      20000331.0 income_return  0.005444  0.005444  0.005361  0.005278   
      20000428.0 income_return  0.005428  0.005428  0.005348  0.005268   
      20000531.0 income_return  0.005609  0.005609  0.005526  0.005444   
...                                  ...       ...       ...       ...   
pwtf  20010831.0 income_return  0.003451  0.003701  0.003784  0.004034   
      20010928.0 income_return  0.002773  0.003097  0.003258  0.003581   
      20011031.0 income_return  0.002530  0.002866  0.003116  0.003367   
      20011130.0 income_return  0.002935  0.003258  0.003420  0.003742   
      20011231.0 income_return  0.003200  0.003617  0.003784  0.004117   

                                  10.0YR    15.0YR    20.0YR    30.0YR  
ctype quotedate  return_type                                            
pwcf  20000131.0 income_return  0.005692  0.005692  0.005609  0.005444  
      20000229.0 income_return  0.005169  0.005092  0.005014  0.004860  
      20000331.0 income_return  0.005196  0.005196  0.005196  0.004947  
      20000428.0 income_return  0.005188  0.005108  0.005108  0.004948  
      20000531.0 income_return  0.005444  0.005361  0.005361  0.005113  
...                                  ...       ...       ...       ...  
pwtf  20010831.0 income_return  0.004283  0.004533  0.004616  0.004533  
      20010928.0 income_return  0.003984  0.004306  0.004466  0.004386  
      20011031.0 income_return  0.003784  0.004117  0.004283  0.004117  
      20011130.0 income_return  0.004065  0.004466  0.004627  0.004306  
      20011231.0 income_return  0.004450  0.004782  0.004864  0.004616  

[72 rows x 12 columns]

In [12]:
imp.reload(dates)
imp.reload(pv)
imp.reload(pvc)
imp.reload(inputs)
imp.reload(outputs)
imp.reload(pzb)
imp.reload(util)
imp.reload(output_to)

<module 'output_to_latexpdf' from 'c:\\Users\\zhang\\OneDrive\\Documents\\GitHub\\UST-yieldcurves_2024\\curve_utils\\src\\development\\output_to_latexpdf.py'>

In [13]:
ret_df = pzb.return_wrapper(df_curve, table_breaks_yr, twostep=False, parmflag=True, padj=False)

c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,4] = "A/A"
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'eomyes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,5] = 'eomyes'
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly c

In [14]:
imp.reload(dates)
imp.reload(pv)
imp.reload(pvc)
imp.reload(inputs)
imp.reload(outputs)
imp.reload(pzb)
imp.reload(util)
imp.reload(output_to)

<module 'output_to_latexpdf' from 'c:\\Users\\zhang\\OneDrive\\Documents\\GitHub\\UST-yieldcurves_2024\\curve_utils\\src\\development\\output_to_latexpdf.py'>

In [15]:
ret_df = pzb.return_wrapper(df_curve, table_breaks_yr, twostep=False, parmflag=True, padj=False)

c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,4] = "A/A"
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'eomyes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,5] = 'eomyes'
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly c

In [16]:
total_ret, income_ret, capgain_ret = seperate_returns_wrapper(ret_df)

KeyError: 'capgain_return'

In [17]:
pzb.calc_capgain_income_ret(curve_df, table_breaks_yr, twostep=False, parmflag=True, padj=False)

c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,4] = "A/A"
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'eomyes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,5] = 'eomyes'
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly c

In [18]:
b = pzb.calc_capgain_income_ret(curve_df, table_breaks_yr, twostep=False, parmflag=True, padj=False)

c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,4] = "A/A"
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'eomyes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  parms_zero.iloc[:,5] = 'eomyes'
c:\Users\zhang\OneDrive\Documents\GitHub\UST-yieldcurves_2024\curve_utils\src\development\parzeroBond.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly c

In [19]:
total_ret = total_ret.reset_index()
income_ret = income_ret.reset_index()


NameError: name 'income_ret' is not defined